In [1]:
import requests
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import re
import nltk

In [2]:
# !pip install text-generation


In [3]:
from transformers import pipeline

# Initialize the text generation pipeline with the desired model
generator = pipeline('text-generation', model='gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:

prompt = "machine learning"
generated_text1 = generator("machine learning", max_length=50)
generated_text2 = generator("World War", max_length=50)
generated_text3 = generator("cancer", max_length=50)

# generated_text1

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
print(generated_text1[0]['generated_text'])
print(generated_text2[0]['generated_text'])
print(generated_text3[0]['generated_text'])
generated_text1

machine learning algorithms are still a decade away, but new research suggests that this is not the case.

The study, published online in Science Advances, found that if a machine learns more highly than humans, it can improve its performance. However
World War 3 - Remastered The Typing of Red Giny's Pixel-Fix The Novel The Pirate's Curse The Pit And The Pendulum The Princess' Heart The Promised Land The Purge Day The Purring Quest The Quest The Quest for Achievements
cancer can make us go crazy…

What did you see this morning?

My eyes, the blue glow of sunlight and our eyes got really warm today.

How cool are they?

Well, I'm going to wait


[{'generated_text': 'machine learning algorithms are still a decade away, but new research suggests that this is not the case.\n\nThe study, published online in Science Advances, found that if a machine learns more highly than humans, it can improve its performance. However'}]

In [6]:
docs = [generated_text1[0]['generated_text'], generated_text2[0]['generated_text'], generated_text3[0]['generated_text']]

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))


cleaned_corpus = []
for document in docs:
    document = document.lower()

    document = re.sub(r'[^\w\s]|[\d]', '', document)

    words = nltk.word_tokenize(document)

    filtered_words = [word for word in words if word.lower() not in stop_words]

    cleaned_sentence = ' '.join(filtered_words)
    cleaned_corpus.append(cleaned_sentence)

cleaned_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['machine learning algorithms still decade away new research suggests case study published online science advances found machine learns highly humans improve performance however',
 'world war remastered typing red ginys pixelfix novel pirates curse pit pendulum princess heart promised land purge day purring quest quest quest achievements',
 'cancer make us go crazy see morning eyes blue glow sunlight eyes got really warm today cool well im going wait']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(cleaned_corpus)

In [10]:
print(type(tf_idf_vector), tf_idf_vector.shape)
# <class'scipy.sparse.csr.csr_matrix'>(3,14)

<class 'scipy.sparse._csr.csr_matrix'> (3, 63)


In [11]:
tf_idf_array = tf_idf_vector.toarray()

print(tf_idf_array)

[[0.         0.2        0.2        0.2        0.         0.
  0.2        0.         0.         0.         0.         0.2
  0.         0.2        0.         0.         0.         0.
  0.         0.         0.2        0.2        0.2        0.
  0.2        0.         0.2        0.2        0.4        0.
  0.         0.2        0.         0.2        0.         0.2
  0.         0.         0.         0.         0.         0.2
  0.         0.         0.         0.         0.         0.
  0.2        0.2        0.         0.2        0.2        0.2
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.18569534 0.         0.         0.         0.         0.
  0.         0.         0.         0.18569534 0.18569534 0.
  0.         0.         0.18569534 0.         0.         0.
  0.         0.18569534 0.         0.         0.         0.
  0.         0.18569534 0.         0.         0.         0.
  0.         0.         0.18569534 0.         0.18569534 0.


In [12]:
words_set = tr_idf_model.get_feature_names_out()
print(words_set)

['achievements' 'advances' 'algorithms' 'away' 'blue' 'cancer' 'case'
 'cool' 'crazy' 'curse' 'day' 'decade' 'eyes' 'found' 'ginys' 'glow' 'go'
 'going' 'got' 'heart' 'highly' 'however' 'humans' 'im' 'improve' 'land'
 'learning' 'learns' 'machine' 'make' 'morning' 'new' 'novel' 'online'
 'pendulum' 'performance' 'pirates' 'pit' 'pixelfix' 'princess' 'promised'
 'published' 'purge' 'purring' 'quest' 'really' 'red' 'remastered'
 'research' 'science' 'see' 'still' 'study' 'suggests' 'sunlight' 'today'
 'typing' 'us' 'wait' 'war' 'warm' 'well' 'world']


In [13]:
import pandas as pd

df_tf_idf = pd.DataFrame(tf_idf_array, columns = words_set)

df_tf_idf

,achievements,advances,algorithms,away,blue,cancer,case,cool,crazy,curse,...,suggests,sunlight,today,typing,us,wait,war,warm,well,world
0,0.000000,0.2,0.2,0.2,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,...,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.185695,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.185695,...,0.0,0.000000,0.000000,0.185695,0.000000,0.000000,0.185695,0.000000,0.000000,0.185695
2,0.000000,0.0,0.0,0.0,0.208514,0.208514,0.0,0.208514,0.208514,0.000000,...,0.0,0.208514,0.208514,0.000000,0.208514,0.208514,0.000000,0.208514,0.208514,0.000000


##Computing TF-IDF from scratch


In [14]:
import numpy as np

n_docs = len(cleaned_corpus)         #·Number of documents in the corpus
n_words_set = len(words_set) #·Number of unique words in the

df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), columns=words_set)

# Compute Term Frequency (TF)
for i in range(n_docs):
    words = cleaned_corpus[i].replace('\n','').split(' ') # Words in the document
    for w in words:
        df_tf[w][i] = df_tf[w][i] + (1 / len(words))

df_tf

,achievements,advances,algorithms,away,blue,cancer,case,cool,crazy,curse,...,suggests,sunlight,today,typing,us,wait,war,warm,well,world
0,0.000000,0.043478,0.043478,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,...,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478
2,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000,...,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000


In [15]:
print("IDF of: ")

idf = {}

for w in words_set:
    k = 0    # number of documents in the corpus that contain this word

    for i in range(n_docs):
        if w in cleaned_corpus[i].split():
            k += 1

    idf[w] =  np.log10(n_docs / k)

    print(f'{w:>15}: {idf[w]:>10}' )

IDF of: 
   achievements: 0.47712125471966244
       advances: 0.47712125471966244
     algorithms: 0.47712125471966244
           away: 0.47712125471966244
           blue: 0.47712125471966244
         cancer: 0.47712125471966244
           case: 0.47712125471966244
           cool: 0.47712125471966244
          crazy: 0.47712125471966244
          curse: 0.47712125471966244
            day: 0.47712125471966244
         decade: 0.47712125471966244
           eyes: 0.47712125471966244
          found: 0.47712125471966244
          ginys: 0.47712125471966244
           glow: 0.47712125471966244
             go: 0.47712125471966244
          going: 0.47712125471966244
            got: 0.47712125471966244
          heart: 0.47712125471966244
         highly: 0.47712125471966244
        however: 0.47712125471966244
         humans: 0.47712125471966244
             im: 0.47712125471966244
        improve: 0.47712125471966244
           land: 0.47712125471966244
       learning: 0.4771212547

In [17]:
df_tf_idf = df_tf.copy()

for w in words_set:
    for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

df_tf_idf

,achievements,advances,algorithms,away,blue,cancer,case,cool,crazy,curse,...,suggests,sunlight,today,typing,us,wait,war,warm,well,world
0,0.000000,0.043478,0.043478,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,...,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478
2,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000,...,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000
